### Installation

In [1]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.8

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
models_name = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Llama-3.2-3B-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    # not pre quantized models
    "unsloth/Meta-Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.2-1B",
] # More models at https://huggingface.co/unsloth



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models_name[2],  #qui stiamo caricando un modello quantizzato a 4bit
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = userdata.get('HF_LLAMA')  you need to request access to Llama models and use here your huggingface key
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

**LoRA (Low-Rank Adaptation)** is a technique used to fine-tune large language models efficiently. Instead of updating all model weights, it adds small trainable matrices to certain layers, drastically reducing the number of parameters to train. This makes fine-tuning faster, cheaper, and requires less data.

In **LoRA**, the added matrix is built using a **low-rank decomposition**:

* Instead of learning a full weight update matrix **ΔW** (which is large), LoRA approximates it as:
  **ΔW ≈ A × B**,
  where:

  * **A** has shape *(original\_dim, r)*
  * **B** has shape *(r, input\_dim)*
  * **r** is a small number (the **rank**), usually much smaller than the original dimensions

This means:

* Only **A** and **B** are trained (not the full matrix),
* The product **A × B** is added to the frozen original weights during training,
* This approach reduces the number of trainable parameters from **O(n²)** to **O(n × r)**.

This low-rank trick makes LoRA both efficient and effective.

The **`lora_alpha`** parameter is a **scaling factor** used in LoRA to control the impact of the low-rank update.

After computing the low-rank approximation $\Delta W = A \times B$, the scaled update is:

$$
\Delta W_{\text{scaled}} = \frac{\alpha}{r} \cdot (A \times B)
$$

Where:

* **$\alpha$** is **`lora_alpha`**
* **$r$** is the **rank** (size of the bottleneck)
* $\frac{\alpha}{r}$ ensures the update has the right magnitude

### Purpose:

* It balances how much the LoRA module influences the model during training.
* Higher `lora_alpha` → stronger effect of the LoRA update.
* Lower `lora_alpha` → weaker effect, more conservative adaptation.

This allows fine control without retraining everything.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!



In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],   #il modello degli embedding portarlo quando si adatta a un dominio specifico o a una nuova lingua
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

#in genere massimo 10 epoche bastano. Salvare ad ogni epoca e poi prendere il migliore

Unsloth: Already have LoRA adapters! We shall skip this step.


In [6]:
!git clone https://huggingface.co/datasets/crux82/squad_it

Cloning into 'squad_it'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 59 (delta 0), reused 0 (delta 0), pack-reused 53 (from 1)
Unpacking objects: 100% (59/59), 15.15 KiB | 969.00 KiB/s, done.
Filtering content: 100% (2/2), 13.16 MiB | 9.22 MiB/s, done.


<a name="Data"></a>
### Data Prep
We now use the SQUAD-it dataset from [Roma Tor Vergata](https://huggingface.co/datasets/crux82/squad_it), which is s derived from the SQuAD dataset and it is obtained through semi-automatic translation of the SQuAD dataset into Italian. You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

In [9]:
#il modello base non ha mai visto prompt, quindi possiamo crearlo noi


inst_prompt = """Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
{}

### Contesto:
{}

### Risposta:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):   #cicla sugli esempi del dataset e formatta bene
    questions = examples["question"]
    contexts  = examples["context"]
    answers   = [e["text"][0] for e in examples["answers"]]
    texts = []
    for questions, contexts, answers in zip(questions, contexts, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = inst_prompt.format(questions, contexts, answers) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("./squad_it/data", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [10]:
print(dataset[0]["text"])

Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
In quale anno si è verificato il terremoto nel Sichuan?

### Contesto:
Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.

### Risposta:
2008<|end_of_text|>


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 100 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [11]:
import xformers
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
#trl libreria di huggingface, la stiamo usando insieme ad unsloth
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,   #unsloth la gestisce in maniera dinamica
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.  (permette di concatenare piu esempi di training insieme, trianing piu veloce ma esempi MOOOLTO corti, fa schifo)
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        #un batch sarà da 16 quindi
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,   #i passaggi di training
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/54159 [00:00<?, ? examples/s]

In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.457 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 54,159 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
50,1.972800
100,1.848600


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

340.8959 seconds used for training.
5.68 minutes used for training.
Peak reserved memory = 2.705 GB.
Peak reserved memory for training = 1.248 GB.
Peak reserved memory % of max memory = 18.35 %.
Peak reserved memory for training % of max memory = 8.466 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [22]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    inst_prompt.format(
        "Quando è iniziata la crisi petrolifera del 1973?", # question
        "La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Di seguito c\'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.\n\n### Domanda:\nQuando è iniziata la crisi petrolifera del 1973?\n\n### Contesto:\nLa crisi petrolifera del 1973 iniziò nell\'ottobre 1973 quando i membri dell\'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell\'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell\'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L\'embargo ha causato una crisi petrolifera, o "shock", con molti effetti a breve e lungo termine sulla politica globale e sull\' economia globale. Più tardi fu chiamato il "primo shock petrolifero", seguito dalla crisi petrolifera del 1979, definita il "secondo shock petrolifero".\n\n### Risposta:\nottobre 1973<|end_of_text|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    inst_prompt.format(
        "Quando è iniziata la crisi petrolifera del 1973?", # question
        "La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Quando è iniziata la crisi petrolifera del 1973?

### Contesto:
La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o "shock", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il "primo shock petrolifero", seguito dalla crisi petrolifera del 1979, definita il "secondo shock petrolifero".

### Risposta:
ottobre 1973<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model.

In [24]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
#QUESTE DUE ISTRUZIONI SALVANO SOLO L ADAPTER


# for saving the merged model (model + adapter)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_4bit",)

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [25]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING, nome della cartella (nel config dell adapter sta il modello base, quindi se lo carica da solo)
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    inst_prompt.format(
        "Quando è iniziata la crisi petrolifera del 1973?", # question
        "La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Quando è iniziata la crisi petrolifera del 1973?

### Contesto:
La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 doll

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [26]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")